In [26]:
from helper import *
import pandas as pd
import rpy2.robjects as ro
from rpy2.robjects import pandas2ri,numpy2ri
from rpy2.robjects.conversion import localconverter
pandas2ri.activate()
import pandas as pd
import os 
import pickle
import numpy as np

In [ ]:
inla

In [2]:
ro.r("""source('helper.R')""")

R[write to console]: Loading required package: Matrix

R[write to console]: Loading required package: sp

R[write to console]: This is INLA_24.04.01 built 2024-04-01 19:43:22 UTC.
 - See www.r-inla.org/contact-us for how to get help.
 - List available models/likelihoods/etc with inla.list.models()
 - Use inla.doc(<NAME>) to access documentation



In [185]:


max_id=get_max("Log")
logger_path=os.path.join("Log",f"{max_id}.plot")


path="/home/jianj0c/project/STP/notebook_SPDE/stochasticVolatility/Log/7.fitting/fits"
results = []
for file in os.listdir(path):
     fit_path=os.path.join(path,file)
     ro.r(f"""

          xx<-readRDS("{fit_path}")
          result<-parse_result(xx$fit,xx$family)
          """)
     df=ro.r("result$df")
     likelihood=ro.r("unlist(result$like)")
     family={"model":ro.r("xx$family$model")[0],
             "Y":ro.r("xx$family$Y")[0]}
     startValue={"prior.std.dev.nominal":ro.r("xx$start_value$prior.std.dev.nominal")[0],
                    "prior.range.nominal":ro.r("xx$start_value$prior.range.nominal")[0],
                    "rspde.order":ro.r("xx$start_value$rspde.order")[0]}
     
     results.append({"df":ro.conversion.rpy2py(df),
                     "likelihood":likelihood,
                     "family":family,
                     "startValue":startValue,
                     "id":file.split("_")[1][:-4]})

In [190]:
models=dict()
for result in results:
    key=f"{result['family']['model']}_{result['family']['Y']}"
    if key not in models:
        models[key]=list()
    models[key].append(result)

save the parsed result

In [187]:

pickle.dump(models,open("/home/jianj0c/project/STP/notebook_SPDE/stochasticVolatility/Log/7.fitting/models.pkl","wb"))

In [208]:
start_values_likelihood={}
model_key="rough_stochvolln"
models[model_key]=sorted(models[model_key],key=lambda x: f"""({x["startValue"]["prior.std.dev.nominal"]:.0f},{x["startValue"]["prior.range.nominal"]:.0f},{x["startValue"]["rspde.order"]:.0f})""")
for fit in models[model_key]:
    
    statValue=fit["startValue"]
    key=f"""({statValue["prior.std.dev.nominal"]:.0f},{statValue["prior.range.nominal"]:.0f},{statValue["rspde.order"]:.0f})"""
    start_values_likelihood[key]=fit["likelihood"]
    
df=pd.DataFrame(start_values_likelihood)
df.index=df.index+1
df = df.style.format(decimal='.', precision=2)
print(df.to_latex())

\begin{tabular}{lrrrrrrrrr}
 & (2,1,3) & (2,1,5) & (2,1,7) & (20,10,3) & (20,10,5) & (20,10,7) & (50,30,3) & (50,30,5) & (50,30,7) \\
1 & -6547921804.53 & -48903015824.43 & -5690153941.15 & 5497.48 & 5496.46 & 8559.26 & 5205.48 & 5212.57 & 5213.07 \\
2 & -47952343183.42 & -7847609152.63 & -6393947941.91 & 5497.49 & 8455.43 & 4869.96 & 5205.50 & 5212.54 & 5205.32 \\
3 & -956397024192.59 & -6260540071.12 & -2451681978.97 & 8342.97 & 8262.06 & 8559.94 & 5205.89 & 5212.54 & 5213.07 \\
4 & -60986010212.44 & -7681007070.07 & -2081003901.68 & 8392.48 & 5496.46 & 8549.35 & 5205.18 & 5203.22 & 5213.39 \\
5 & -1187222870003.98 & -61631736458.34 & -6229785438.22 & 8342.97 & 5496.46 & 8559.65 & 5294.71 & 5212.10 & 5213.05 \\
6 & -45859408711.41 & -36300903214.72 & -6275629451.94 & 8342.97 & 5496.46 & 4315.93 & 5205.37 & 5212.54 & 5213.06 \\
7 & -27225559698.12 & -8649986367.51 & -9584260954.73 & 8392.48 & 8532.48 & 8561.88 & 5295.67 & 5212.11 & 5213.07 \\
8 & -10679535078.77 & -23137315231.95 & -5

In [209]:
model_key="rough_stochvolln"
max_id_=np.argmax([max(x["likelihood"]) for x in models[model_key] ])
print("config id",models[model_key][max_id_]["id"])
print("run id",np.argmax(models[model_key][max_id_]["likelihood"]))

config id 33
run id 6


## gather all models result into one Table

In [245]:
records=[]
for model_key in ["rough_stochvol"]:#,"rough_stochvol.t","rough_stochvol.nig","rough_stochvolln"]:
    max_id_=np.argmax([max(x["likelihood"]) for x in models[model_key] ])
    fits_for_a_start_values=models[model_key][max_id_]
    idd=np.argmax(fits_for_a_start_values["likelihood"])
    df=models[model_key][max_id_]["df"]
    tmp_df=df.loc[[x for x in df.index if x.startswith(f"e{idd}")]]
    new_key=model_key.replace("_","-")
    indexx=[f"{new_key}_{x[2:]}" for x in tmp_df.index]
    tmp_df.index=indexx
    records.append(tmp_df)

    

In [246]:
models[model_key][max_id_]["df"]

,mean,sd,0.025quant,0.5quant,0.975quant,mode
e1std.dev,121.844000,2.021220,117.798000,121.851000,125.797000,121.936000
e1range,1.394610,0.084387,1.237600,1.390140,1.571880,1.371880
e1nu,0.074124,0.003972,0.067004,0.073876,0.082559,0.073131
e2std.dev,50.565100,1.336070,48.074600,50.502500,53.294900,50.188600
e2range,65.190300,2.576830,60.484700,65.057200,70.690400,64.636200
e2nu,0.036617,0.002454,0.032385,0.036383,0.041955,0.035791
e3std.dev,137.707000,0.641998,136.261000,137.795000,138.736000,138.047000
e3range,13.403300,0.046555,13.307100,13.405000,13.492600,13.415400
e3nu,0.767853,0.002012,0.763281,0.768135,0.770960,0.769250
e4std.dev,55.487200,1.501690,52.724100,55.430200,58.637400,55.548200


In [240]:
df=pd.concat(records).reset_index()
df = df.style.format(decimal='.', precision=5)
df

,index,mean,sd,0.025quant,0.5quant,0.975quant,mode
0,rough-stochvol_std.dev,127.86200,2.70749,122.35200,127.95900,133.05500,128.22300
1,rough-stochvol_range,1.54713,0.14920,1.26703,1.54180,1.85198,1.52458
2,rough-stochvol_nu,0.10860,0.00972,0.08936,0.10880,0.12733,0.10994
3,rough-stochvol.t_std.dev,0.70362,0.07292,0.58123,0.69669,0.86914,0.68687
4,rough-stochvol.t_range,26.52500,6.47911,15.53620,25.92620,40.63260,24.88210
5,rough-stochvol.t_nu,0.23573,0.04821,0.15610,0.23045,0.34472,0.21969
6,rough-stochvol.nig_std.dev,0.71715,0.13988,0.51887,0.69176,1.06060,0.64626
7,rough-stochvol.nig_range,26.16960,8.56542,12.96770,24.93090,46.76570,22.99130
8,rough-stochvol.nig_nu,0.25164,0.08804,0.13016,0.23387,0.47109,0.20054
9,rough-stochvolln_std.dev,26.78780,0.13430,26.51970,26.79090,27.04560,26.80100


In [243]:
df=pd.concat(records).reset_index()
#df = df.style.format(decimal='.', precision=5)
print(df.to_latex())

\begin{tabular}{llrrrrrr}
\toprule
 & index & mean & sd & 0.025quant & 0.5quant & 0.975quant & mode \\
\midrule
0 & rough-stochvol_std.dev & 127.862000 & 2.707490 & 122.352000 & 127.959000 & 133.055000 & 128.223000 \\
1 & rough-stochvol_range & 1.547130 & 0.149202 & 1.267030 & 1.541800 & 1.851980 & 1.524580 \\
2 & rough-stochvol_nu & 0.108601 & 0.009720 & 0.089362 & 0.108795 & 0.127326 & 0.109941 \\
3 & rough-stochvol.t_std.dev & 0.703617 & 0.072924 & 0.581232 & 0.696687 & 0.869138 & 0.686868 \\
4 & rough-stochvol.t_range & 26.525000 & 6.479110 & 15.536200 & 25.926200 & 40.632600 & 24.882100 \\
5 & rough-stochvol.t_nu & 0.235727 & 0.048211 & 0.156098 & 0.230452 & 0.344719 & 0.219692 \\
6 & rough-stochvol.nig_std.dev & 0.717155 & 0.139878 & 0.518869 & 0.691763 & 1.060600 & 0.646259 \\
7 & rough-stochvol.nig_range & 26.169600 & 8.565420 & 12.967700 & 24.930900 & 46.765700 & 22.991300 \\
8 & rough-stochvol.nig_nu & 0.251638 & 0.088044 & 0.130160 & 0.233874 & 0.471094 & 0.200544 \\
9 & rou